# STIR: Polynomial Commitment

In [20]:
from sage.rings.finite_rings.all import *
from sage.rings.polynomial.all import *
from sage.misc.prandom import sample

In [21]:
def chunk_poly(poly, split_degree):
    splits = [ poly.parent()(0) for _ in range(split_degree) ]
    indeterminate = poly.parent().gen()

    for (i,coef) in enumerate(poly.list()):
        ndx = i % split_degree
        deg = i // split_degree
        splits[ndx] = splits[ndx] + coef*(indeterminate**deg)

    pp = poly.parent()(0)

    for (j,mini_poly) in enumerate(splits):
        pp = pp + mini_poly.subs({ indeterminate : indeterminate** split_degree})*(indeterminate**j)
    assert pp == poly, "Polynomial reconstruction failed"

    return splits

def reed_solomon_encode(fx, eval_domain):
    return [fx(x) for x in eval_domain ]

def fold(commit_poly, split_size, challenge, eval_domain):
    """
    K-ary fold of the polynomial and the updated evaluation domain
    """
    splits = chunk_poly(commit_poly, split_size)
    updated_domain = list(set([x**split_size for x in eval_domain]))
    new_poly = commit_poly.parent()(0)
    for i in range(split_size):
        new_poly = new_poly + (challenge**i)*splits[i]

    return (new_poly, updated_domain)

def ood_sample(fq, count, eval_domain):
    """
    Sample random out of domain elements
    """
    result = list()

    while count > 0:
        rng = fq.random_element()
        if rng in eval_domain:
            continue
        result.append(rng)
        count = count - 1

    return result

def ans_vanishing_poly(g_poly, Rout, Rshift):
    indexing_set = list(Rout)
    indexing_set.extend(Rshift)
    interpolant = [(ndx, g_poly(ndx)) for ndx in indexing_set ]
    poly_var = g_poly.parent().gen()
    vanish = prod([(poly_var - root) for root in indexing_set])

    return (g_poly.parent().lagrange_polynomial(interpolant), vanish)    

def poly_quotient(g_poly, Rout, Rshift):
    (ans_poly,vanish) = ans_vanishing_poly(g_poly, Rout, Rshift)
    numerator = g_poly - ans_poly
    assert numerator % vanish == 0
    return numerator // vanish

In [22]:
q = 15*(2**27) + 1 # prime field
fq_star_order = q - 1   
q_smooth_order = 2**27  
fft_order = 2**10 # The size of the RS Codeword
s = 4
Fq = GF(q)
Rx = PolynomialRing(Fq, "X")
Rxy = PolynomialRing(Rx, "X,Y")
X = Rx.gen()
Y = Rxy.gens()[1]
fq_star_gen = Fq(31) # Fq.multiplicative_generator()
ω = fq_star_gen^(fq_star_order // fft_order)
# print(f"ω={ω}")

In [23]:
d = 2**8 # The maximum degree of polynomial
k = 4 # Folding parameter
M = int(ceil(log(d, k)));
kis = list()    # K's at each iteration which is same as k
tis = list()    # tis at each iteration
dis = list()    # dis at each iteration

for i in range(M):
    kis.append(k)
    denom = prod(kis[0:i])
    di = Integer(d//denom)
    dis.append(di)
    tis.append(di-1)

print(f"RS Parameters: ω={ω}, d={d}, k={k}, M={M}, dis={dis}, tis={tis}")

RS Parameters: ω=341742893, d=256, k=4, M=4, dis=[256, 64, 16, 4], tis=[255, 63, 15, 3]


### Intial Function

In [ ]:
f_0 = Rx.random_element(d-1)
S_0 = [ ω**i for i in  range(fft_order) ]
code_word_0 = reed_solomon_encode(f_0, S_0);
r_0_fold = Fq(293327713) # Fq.random_element()
print(f"|S_0|={len(S_0)}\nf0={f_0}\nr_0_fold={r_0_fold}\n")

(g_1, S_1) = fold(f_0, k, r_0_fold, S_0)

code_word_1 = reed_solomon_encode(g_1, S_1)
print(f"|S_1|={len(S_1)}\ng_1={g_1}\n")

R1_out = [442668221, 870054799, 1716171781, 1743092226] # ood_sample(Fq, s, S_1)
beta_1 = [g_1(x) for x in R1_out]
# print(beta_1)

R1_fold = Fq.random_element()
R1_comb = Fq.random_element()

R1_shift = [970382339, 1713712948, 434423500, 1086923312] # sample(S_0, tis[0])

g_prime = poly_quotient(g_1, R1_out, R1_shift)

print(f"g_prime={g_prime}")
